# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [26]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
# stocks
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# data.status_code
data

{'companyName': 'Apple Inc',
 'marketcap': 2189405390829,
 'week52high': 148.55,
 'week52low': 84.97,
 'week52highSplitAdjustOnly': 149.46,
 'week52lowSplitAdjustOnly': 82.55,
 'week52change': 0.55462082313085,
 'sharesOutstanding': 16992872908,
 'float': 0,
 'avg10Volume': 75276103,
 'avg30Volume': 84270998,
 'day200MovingAvg': 128.49,
 'day50MovingAvg': 129.97,
 'employees': 152637,
 'ttmEPS': 4.59,
 'ttmDividendRate': 0.8559943675631079,
 'dividendYield': 0.006763389087106356,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-26',
 'nextEarningsDate': '2021-07-23',
 'peRatio': 28.51764232163621,
 'beta': 1.5001749688327113,
 'maxChangePercent': 47.91400665922011,
 'year5ChangePercent': 4.522641252162409,
 'year2ChangePercent': 1.8696217325035929,
 'year1ChangePercent': 0.5779957417760948,
 'ytdChangePercent': -0.04989769002905761,
 'month6ChangePercent': 0.03407231377546739,
 'month3ChangePercent': 0.05144695267213974,
 'month1ChangePercent': -0.01407727318672046,
 'day30ChangePe

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.5779957417760948

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)
# final_dataframe
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
#     print(data)
    
    for symbol in symbol_string.split(','):
#         print(symbol)
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['price'], 
                    data[symbol]['stats']['year1ChangePercent'], 
                    'N/A'
                ], 
                index=my_columns), 
            ignore_index=True
        )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,143.90,0.54643,N/A
1,AAL,25.00,0.466784,N/A
2,AAP,201.66,0.381171,N/A
3,AAPL,129.42,0.588517,N/A
4,ABBV,117.26,0.268992,N/A
...,...,...,...,...
500,YUM,119.30,0.274194,N/A
501,ZBH,167.05,0.240879,N/A
502,ZBRA,520.60,0.99076,N/A
503,ZION,58.56,0.628992,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_dataframe.sort_values(by='One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50].reset_index()
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,179,FCX,43.07,3.23364,N/A
1,275,LB,67.73,2.74485,N/A
2,253,IVZ,29.35,2.18609,N/A
3,410,SIVB,603.87,1.67967,N/A
4,148,DVN,32.22,1.67573,N/A
5,208,GPS,32.40,1.6638,N/A
6,314,MOS,36.18,1.62781,N/A
7,441,TPR,44.12,1.61572,N/A
8,344,NUE,112.13,1.59008,N/A
9,174,F,16.56,1.45025,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [69]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of you porfolio:")
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input("Enter the size of you porfolio:")
    


In [70]:
portfolio_input()
print(portfolio_size)

Enter the size of you porfolio:10000000
10000000


In [71]:
position_size = float(portfolio_size)/len(final_dataframe.index)
# position_size
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,179,FCX,43.07,3.23364,4643
1,275,LB,67.73,2.74485,2952
2,253,IVZ,29.35,2.18609,6814
3,410,SIVB,603.87,1.67967,331
4,148,DVN,32.22,1.67573,6207
5,208,GPS,32.40,1.6638,6172
6,314,MOS,36.18,1.62781,5527
7,441,TPR,44.12,1.61572,4533
8,344,NUE,112.13,1.59008,1783
9,174,F,16.56,1.45025,12077


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [105]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile', # HaMeon 
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
# hqm_datatfraem

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ], 
                index=hqm_columns), 
            ignore_index=True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.50,N/A,0.554701,N/A,0.206387,N/A,0.192981,N/A,0.0341467,N/A,N/A
1,AAL,24.80,N/A,0.318539,N/A,0.499732,N/A,0.193286,N/A,0.13192,N/A,N/A
2,AAP,198.18,N/A,0.336238,N/A,0.278658,N/A,0.128924,N/A,-0.051604,N/A,N/A
3,AAPL,127.74,N/A,0.555267,N/A,0.0345086,N/A,0.0394291,N/A,-0.015731,N/A,N/A
4,ABBV,113.61,N/A,0.258359,N/A,0.0738511,N/A,0.0685172,N/A,-0.0302352,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.37,N/A,0.265558,N/A,0.135365,N/A,0.146703,N/A,-0.0154412,N/A,N/A
501,ZBH,161.11,N/A,0.174149,N/A,0.0732246,N/A,0.00567889,N/A,-0.0855644,N/A,N/A
502,ZBRA,518.42,N/A,0.898586,N/A,0.365617,N/A,0.080448,N/A,0.0713055,N/A,N/A
503,ZION,59.91,N/A,0.563047,N/A,0.347207,N/A,0.0494949,N/A,0.00439472,N/A,N/A


In [106]:
hqm_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Ticker                         505 non-null    object 
 1   Price                          505 non-null    float64
 2   Number of Shares to Buy        505 non-null    object 
 3   One-Year Price Return          501 non-null    object 
 4   One-Year Return Percentile     505 non-null    object 
 5   Six-Month Price Return         501 non-null    object 
 6   Six-Month Return Percentile    505 non-null    object 
 7   Three-Month Price Return       501 non-null    object 
 8   Three-Month Return Percentile  505 non-null    object 
 9   One-Month Price Return         501 non-null    object 
 10  One-Month Return Percentile    505 non-null    object 
 11  HQM Score                      505 non-null    object 
dtypes: float64(1), object(11)
memory usage: 47.5+ KB


In [107]:
hqm_dataframe.dropna(inplace=True)
# hqm_dataframe.reset_index(inplace=True)
# df1 = hqm_dataframe[hqm_dataframe.isna().any(axis=1)]
# df1

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [108]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'    
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100 # Calculate percentile "Hameon"
        
hqm_dataframe



,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.50,N/A,0.554701,0.718563,0.206387,0.518962,0.192981,0.786427,0.0341467,0.618762,N/A
1,AAL,24.80,N/A,0.318539,0.451098,0.499732,0.928144,0.193286,0.788423,0.13192,0.962076,N/A
2,AAP,198.18,N/A,0.336238,0.479042,0.278658,0.672655,0.128924,0.526946,-0.051604,0.0998004,N/A
3,AAPL,127.74,N/A,0.555267,0.722555,0.0345086,0.149701,0.0394291,0.209581,-0.015731,0.245509,N/A
4,ABBV,113.61,N/A,0.258359,0.367265,0.0738511,0.219561,0.0685172,0.291417,-0.0302352,0.165669,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.37,N/A,0.265558,0.383234,0.135365,0.381238,0.146703,0.598802,-0.0154412,0.247505,N/A
501,ZBH,161.11,N/A,0.174149,0.205589,0.0732246,0.217565,0.00567889,0.129741,-0.0855644,0.0439122,N/A
502,ZBRA,518.42,N/A,0.898586,0.92016,0.365617,0.820359,0.080448,0.331337,0.0713055,0.826347,N/A
503,ZION,59.91,N/A,0.563047,0.730539,0.347207,0.794411,0.0494949,0.243513,0.00439472,0.393214,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [109]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)   

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.50,N/A,0.554701,0.718563,0.206387,0.518962,0.192981,0.786427,0.0341467,0.618762,0.660679
1,AAL,24.80,N/A,0.318539,0.451098,0.499732,0.928144,0.193286,0.788423,0.13192,0.962076,0.782435
2,AAP,198.18,N/A,0.336238,0.479042,0.278658,0.672655,0.128924,0.526946,-0.051604,0.0998004,0.444611
3,AAPL,127.74,N/A,0.555267,0.722555,0.0345086,0.149701,0.0394291,0.209581,-0.015731,0.245509,0.331836
4,ABBV,113.61,N/A,0.258359,0.367265,0.0738511,0.219561,0.0685172,0.291417,-0.0302352,0.165669,0.260978
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.37,N/A,0.265558,0.383234,0.135365,0.381238,0.146703,0.598802,-0.0154412,0.247505,0.402695
501,ZBH,161.11,N/A,0.174149,0.205589,0.0732246,0.217565,0.00567889,0.129741,-0.0855644,0.0439122,0.149202
502,ZBRA,518.42,N/A,0.898586,0.92016,0.365617,0.820359,0.080448,0.331337,0.0713055,0.826347,0.724551
503,ZION,59.91,N/A,0.563047,0.730539,0.347207,0.794411,0.0494949,0.243513,0.00439472,0.393214,0.540419


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [110]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
# hqm_dataframe.reset_index(inplace=True)
# hqm_dataframe.drop(['index'], axis='columns', inplace=True) #'level_0', 
hqm_dataframe = hqm_dataframe[:50].copy()
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
344,NUE,109.07,N/A,1.48748,0.986028,0.977237,0.998004,0.74986,1,0.179296,0.98004,0.991018
149,DXC,40.76,N/A,1.24966,0.98004,0.622365,0.968064,0.473475,0.996008,0.184222,0.984032,0.982036
174,F,16.05,N/A,1.21201,0.976048,0.717455,0.986028,0.305144,0.96008,0.37608,1,0.980539
148,DVN,32.57,N/A,1.34074,0.982036,0.966026,0.996008,0.248413,0.902196,0.264235,0.994012,0.968563
106,COF,171.98,N/A,1.12004,0.966068,0.809686,0.992016,0.306083,0.962076,0.0885358,0.886228,0.951597
426,SYF,50.50,N/A,1.01861,0.948104,0.551945,0.952096,0.25159,0.914172,0.113317,0.934132,0.937126
249,IRM,46.44,N/A,0.687266,0.824351,0.634483,0.972056,0.29792,0.954092,0.179584,0.982036,0.933134
412,SLB,37.25,N/A,0.7012,0.832335,0.56309,0.954092,0.254647,0.918164,0.201189,0.99002,0.923653
29,AMAT,145.09,N/A,1.36184,0.984032,0.580624,0.958084,0.238898,0.884232,0.0782117,0.860279,0.921657
253,IVZ,30.12,N/A,1.96944,0.996008,0.69827,0.984032,0.250609,0.91018,0.0633437,0.788423,0.919661


In [111]:
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NUE,109.07,N/A,1.48748,0.986028,0.977237,0.998004,0.74986,1,0.179296,0.98004,0.991018
1,DXC,40.76,N/A,1.24966,0.98004,0.622365,0.968064,0.473475,0.996008,0.184222,0.984032,0.982036
2,F,16.05,N/A,1.21201,0.976048,0.717455,0.986028,0.305144,0.96008,0.37608,1,0.980539
3,DVN,32.57,N/A,1.34074,0.982036,0.966026,0.996008,0.248413,0.902196,0.264235,0.994012,0.968563
4,COF,171.98,N/A,1.12004,0.966068,0.809686,0.992016,0.306083,0.962076,0.0885358,0.886228,0.951597
5,SYF,50.50,N/A,1.01861,0.948104,0.551945,0.952096,0.25159,0.914172,0.113317,0.934132,0.937126
6,IRM,46.44,N/A,0.687266,0.824351,0.634483,0.972056,0.29792,0.954092,0.179584,0.982036,0.933134
7,SLB,37.25,N/A,0.7012,0.832335,0.56309,0.954092,0.254647,0.918164,0.201189,0.99002,0.923653
8,AMAT,145.09,N/A,1.36184,0.984032,0.580624,0.958084,0.238898,0.884232,0.0782117,0.860279,0.921657
9,IVZ,30.12,N/A,1.96944,0.996008,0.69827,0.984032,0.250609,0.91018,0.0633437,0.788423,0.919661


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [112]:
portfolio_input()

Enter the size of you porfolio:1000000


In [113]:
position_size  = float(portfolio_size)/len(hqm_dataframe.index)
# position_size
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NUE,109.07,183,1.48748,0.986028,0.977237,0.998004,0.74986,1,0.179296,0.98004,0.991018
1,DXC,40.76,490,1.24966,0.98004,0.622365,0.968064,0.473475,0.996008,0.184222,0.984032,0.982036
2,F,16.05,1246,1.21201,0.976048,0.717455,0.986028,0.305144,0.96008,0.37608,1,0.980539
3,DVN,32.57,614,1.34074,0.982036,0.966026,0.996008,0.248413,0.902196,0.264235,0.994012,0.968563
4,COF,171.98,116,1.12004,0.966068,0.809686,0.992016,0.306083,0.962076,0.0885358,0.886228,0.951597
5,SYF,50.50,396,1.01861,0.948104,0.551945,0.952096,0.25159,0.914172,0.113317,0.934132,0.937126
6,IRM,46.44,430,0.687266,0.824351,0.634483,0.972056,0.29792,0.954092,0.179584,0.982036,0.933134
7,SLB,37.25,536,0.7012,0.832335,0.56309,0.954092,0.254647,0.918164,0.201189,0.99002,0.923653
8,AMAT,145.09,137,1.36184,0.984032,0.580624,0.958084,0.238898,0.884232,0.0782117,0.860279,0.921657
9,IVZ,30.12,664,1.96944,0.996008,0.69827,0.984032,0.250609,0.91018,0.0633437,0.788423,0.919661


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [117]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [3]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

In [2]:
columns_format = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in columns_format.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, columns_format[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', columns_format[column][0], columns_format[column][1])
    

NameError: name 'string_template' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [120]:
writer.save()